### Importação de Bibliotecas e Configurações Iniciais

In [16]:
import os
import re
import json
import pytesseract
import fitz 
from pdf2image import convert_from_path
from PIL import Image
from unidecode import unidecode
import pandas as pd
import api  # Importa a API de um arquivo separado

# Configuração para exibir mais colunas e linhas no Jupyter Notebook
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", 50)

# Configuração do Tesseract OCR (ajuste conforme necessário)
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

# Diretórios para PDFs e resultados
PASTA_PDFS = "notas_fiscais"
PASTA_RESULTADOS = "resultados"
os.makedirs(PASTA_RESULTADOS, exist_ok=True)


### Definição do Template para Extração Inteligente

In [17]:
TEMPLATE = """
Você é um assistente especializado em extrair dados de notas fiscais brasileiras em PDF.
O seguinte texto foi extraído de uma nota fiscal:

{nota_texto}

Sua tarefa é estruturar os dados no seguinte formato JSON:
- "numero_nota": Deve ser o número da nota fiscal. Pode aparecer como "No.", "Número NF" ou "Número NFS".
- "serie": Série da nota fiscal.
- "data_emissao": Data de emissão no formato DD/MM/AAAA ou similar.
- "chave_acesso": Código de 44 dígitos, se disponível.
- "tipo_nota": "Produto" ou "Serviço".
- "cnpj_emitente": CNPJ do emitente.
- "nome_emitente": Nome da empresa emitente.
- "endereco_emitente": Endereço completo do emitente.
- "cnpj_cpf_destinatario": CNPJ ou CPF do destinatário.
- "nome_destinatario": Nome do destinatário.
- "endereco_destinatario": Endereço completo do destinatário.
- "itens": Lista contendo os itens da nota fiscal, com os seguintes campos:
    - "codigo": Código do item.
    - "descricao": Descrição do item.
    - "quantidade": Quantidade comprada.
    - "unidade": Unidade de medida.
    - "valor_unitario": Valor unitário do item.
    - "valor_total": Valor total do item.
- "subtotal": Valor total dos produtos antes dos impostos.
- "impostos": Dicionário contendo:
    - "ICMS", "IPI", "ISS", "PIS", "COFINS" (valores sempre em formato numérico, ex: 0.00).
- "valor_total_nota": Valor total da nota fiscal.
- "forma_pagamento": Método de pagamento.
- "valor_pago": Valor pago pelo cliente.

⚠️ **Atenção**:  
- Responda **exclusivamente** com um JSON válido, **sem nenhum outro texto**.
- Certifique-se de que os valores de impostos e totais estão **em formato numérico** (exemplo: `0.00` em vez de `"0,00"`).
- Caso um campo não exista, defina-o como `null`.
"""


### Funções para Extração de Texto

In [18]:
def extrair_texto_pdf(pdf_path):
    """Extrai texto diretamente de um PDF, se possível."""
    texto_extraido = ""
    with fitz.open(pdf_path) as doc:
        for pagina in doc:
            texto_extraido += pagina.get_text("text") + "\n"
    
    return unidecode(texto_extraido.strip()) if texto_extraido.strip() else None

def extrair_texto_ocr(pdf_path):
    """Converte PDF escaneado em imagem e aplica OCR."""
    imagens = convert_from_path(pdf_path)
    texto_extraido = ""

    for img in imagens:
        texto_ocr = pytesseract.image_to_string(img, lang="por")
        texto_extraido += texto_ocr + "\n"

    return unidecode(texto_extraido.strip())


### 5. Processamento do PDF

In [19]:
def processar_pdf(pdf_path):
    """Processa um PDF e estrutura os dados corretamente."""
    try:
        # Tenta extrair texto diretamente
        texto_extraido = extrair_texto_pdf(pdf_path)

        # Se falhar, usa OCR
        if texto_extraido is None:
            print(f"🔹 O PDF '{pdf_path}' parece ser escaneado. Aplicando OCR...")
            texto_extraido = extrair_texto_ocr(pdf_path)

        # Criar prompt para IA
        prompt = TEMPLATE.format(nota_texto=texto_extraido)

        # Enviar para modelo Gemini
        response = api.model.generate_content(prompt)

        # Processar a resposta da API Gemini
        resposta_ia = response.text.strip()

        # Remover blocos de código Markdown
        resposta_ia = re.sub(r"```json\n(.*?)\n```", r"\1", resposta_ia, flags=re.DOTALL).strip()

        # Garantir que a resposta seja JSON válido
        try:
            dados_extraidos = json.loads(resposta_ia)
        except json.JSONDecodeError:
            print(f"❌ Erro ao processar JSON da resposta: {resposta_ia}")
            dados_extraidos = {"erro": "Resposta inválida da API"}

        return {
            "arquivo": os.path.basename(pdf_path),
            "conteudo_bruto": texto_extraido,
            "dados_extraidos": dados_extraidos
        }
    except Exception as e:
        return {"arquivo": os.path.basename(pdf_path), "erro": str(e)}


### Processamento em Lote e Leitura do JSON

In [20]:
def processar_todos_pdfs():
    """Processa todos os PDFs da pasta e salva os resultados em JSON."""
    arquivos = [f for f in os.listdir(PASTA_PDFS) if f.lower().endswith(".pdf")]

    resultados = []
    for arquivo in arquivos:
        caminho_arquivo = os.path.join(PASTA_PDFS, arquivo)
        print(f"🔹 Processando: {arquivo}")
        resultado = processar_pdf(caminho_arquivo)
        resultados.append(resultado)

    # Salvar JSON consolidado
    json_path = os.path.join(PASTA_RESULTADOS, "notas_extraidas.json")
    with open(json_path, "w", encoding="utf-8") as json_file:
        json.dump(resultados, json_file, ensure_ascii=False, indent=4)

    print(f"✅ Processamento concluído! Dados salvos em {json_path}")

# Executar Processamento
processar_todos_pdfs()

🔹 Processando: Teste-02 - Copia (2).pdf
🔹 Processando: Teste-02.pdf
🔹 Processando: Teste-03 - Copia (2).pdf
🔹 Processando: Teste-03 - Copia (3).pdf
🔹 Processando: Teste-03 - Copia (4).pdf
🔹 Processando: Teste-03.pdf
🔹 Processando: Teste-04 - Copia (10).PDF
🔹 Processando: Teste-04 - Copia.PDF
🔹 Processando: Teste-04.PDF
✅ Processamento concluído! Dados salvos em resultados\notas_extraidas.json


### Caminho do JSON

In [21]:
json_path = r"C:\Users\breno\OneDrive\Documentos\Python_Projects\ianf_agent\resultados\notas_extraidas.json"

### Ler Arquivos Json

In [22]:
# 1. Verificar se o arquivo existe
if not os.path.exists(json_path):
    print(f"❌ Arquivo não encontrado: {json_path}")
else:
    # 2. Carregar os dados do JSON
    with open(json_path, "r", encoding="utf-8") as file:
        dados_json = json.load(file)

    # 3. Converter para DataFrame
    df = pd.DataFrame(dados_json)

    # 4. Expandir a coluna "dados_extraidos" para separar os detalhes
    if "dados_extraidos" in df.columns:
        df_detalhado = df["dados_extraidos"].apply(pd.Series)

        # 5. Concatenar os dados no mesmo DataFrame
        df = pd.concat([df.drop(columns=["dados_extraidos"]), df_detalhado], axis=1)

    # 6. Renomear colunas para português
    df.rename(columns={
        "arquivo": "Arquivo",
        "conteudo_bruto": "Conteúdo Bruto",
        "Number": "Número da Nota",
        "Series": "Série",
        "EmissionDate": "Data de Emissão",
        "AccessKey": "Chave de Acesso",
        "Type": "Tipo de Nota",
        "EmitterCnpj": "CNPJ Emitente",
        "EmitterName": "Nome Emitente",
        "EmitterAddress": "Endereço Emitente",
        "ReceiverCnpj": "CNPJ Destinatário",
        "ReceiverName": "Nome Destinatário",
        "ReceiverAddress": "Endereço Destinatário",
        "Items": "Itens",
        "Subtotal": "Subtotal",
        "Taxes": "Impostos",
        "TotalValue": "Valor Total",
        "PaymentForm": "Forma de Pagamento",
        "PaidValue": "Valor Pago"
    }, inplace=True)

    # 7. Expandir endereços do emitente e destinatário
    if "Endereço Emitente" in df.columns:
        df_end_emitente = df["Endereço Emitente"].apply(pd.Series)
        df = pd.concat([df.drop(columns=["Endereço Emitente"]), df_end_emitente], axis=1)

    if "Endereço Destinatário" in df.columns:
        df_end_destinatario = df["Endereço Destinatário"].apply(pd.Series)
        df = pd.concat([df.drop(columns=["Endereço Destinatário"]), df_end_destinatario], axis=1)

    # 8. Expandir impostos
    if "Impostos" in df.columns:
        df_impostos = df["Impostos"].apply(pd.Series)
        df = pd.concat([df.drop(columns=["Impostos"]), df_impostos], axis=1)

    # 9. Expandir os detalhes de ICMS, IPI, ISS, PIS, COFINS (se existirem)
    impostos_detalhados = ["ICMS", "IPI", "ISS", "PIS", "COFINS"]
    for imposto in impostos_detalhados:
        if imposto in df.columns:
            df_imposto = df[imposto].apply(pd.Series)
            df = pd.concat([df.drop(columns=[imposto]), df_imposto.add_prefix(f"{imposto} ")], axis=1)

    # 10. Exibir as 5 primeiras linhas do DataFrame
    display(df.head(40))

    print("✅ Dados carregados e estruturados com sucesso!")

,Arquivo,Conteúdo Bruto,numero_nota,serie,data_emissao,chave_acesso,tipo_nota,cnpj_emitente,nome_emitente,endereco_emitente,cnpj_cpf_destinatario,nome_destinatario,endereco_destinatario,itens,subtotal,impostos,valor_total_nota,forma_pagamento,valor_pago
0,Teste-02 - Copia (2).pdf,SERIE:\nN:\nRECEBEDOR: FRANCISCO BRENO SANDRO ...,000000706,1,30/07/2021,23210740572126000155550010000007061000006017,Produto,40.572.126/0001-55,MONOBLOCO SERV DE MANUTENCAO E REPARACAO MEC F...,"AV HERACLITO GRACA 1001, --- ALDEOTA FORTALEZA...",603.998.143-10,FRANCISCO BRENO SANDRO GABRIEL DE SOUZA,RUA OSCAR SOARES BARROS - SN TAIBA 62670-000 S...,"[{'codigo': '56008', 'descricao': 'MANGUEIRA R...",22.00,"{'ICMS': 0.0, 'IPI': 0.0, 'ISS': None, 'PIS': ...",22.00,None,NaN
1,Teste-02.pdf,SERIE:\nN:\nRECEBEDOR: FRANCISCO BRENO SANDRO ...,000000706,1,30/07/2021,23210740572126000155550010000007061000006017,Produto,40.572.126/0001-55,MONOBLOCO SERV DE MANUTENCAO E REPARACAO MEC F...,"AV HERACLITO GRACA 1001, --- ALDEOTA FORTALEZA...",603.998.143-10,FRANCISCO BRENO SANDRO GABRIEL DE SOUZA,RUA OSCAR SOARES BARROS - SN TAIBA 62670-000 S...,"[{'codigo': '56008', 'descricao': 'MANGUEIRA R...",22.00,"{'ICMS': 0.0, 'IPI': 0.0, 'ISS': None, 'PIS': ...",22.00,None,NaN
2,Teste-03 - Copia (2).pdf,NOTA FISCAL DE SERVICO ELETRONICA - NFS-e\nFRA...,3,None,01/03/2023 07:35:01,None,Serviço,48.244.593/0001-68,FRANCISCO BRENO SANDRO GABRIEL BARROS 60399814310,"R OSCAR SOARES BARROS, 0, TAIBA, SAO GONCALO D...",38.730.525/0001-55,BS PARC PACATUBA EMPREENDIMENTO IMOBILIARIO LTDA,"AV DOUTOR MENDEL STEINBRUCH,KM 12, - PAVUNA, P...","[{'codigo': '7.02', 'descricao': 'Servicos pre...",4400.00,"{'ICMS': None, 'IPI': None, 'ISS': 0.0, 'PIS':...",4400.00,None,NaN
3,Teste-03 - Copia (3).pdf,NOTA FISCAL DE SERVICO ELETRONICA - NFS-e\nFRA...,3,None,01/03/2023 07:35:01,None,Serviço,48.244.593/0001-68,FRANCISCO BRENO SANDRO GABRIEL BARROS,"R OSCAR SOARES BARROS, 0, TAIBA, SAO GONCALO D...",38.730.525/0001-55,BS PARC PACATUBA EMPREENDIMENTO IMOBILIARIO LTDA,"AV DOUTOR MENDEL STEINBRUCH,KM 12, - PAVUNA, P...","[{'codigo': '7.02', 'descricao': 'Servicos pre...",NaN,"{'ICMS': None, 'IPI': None, 'ISS': 0.0, 'PIS':...",4400.00,None,4400.0
4,Teste-03 - Copia (4).pdf,NOTA FISCAL DE SERVICO ELETRONICA - NFS-e\nFRA...,3,None,01/03/2023 07:35:01,None,Serviço,48.244.593/0001-68,FRANCISCO BRENO SANDRO GABRIEL BARROS 60399814310,"R OSCAR SOARES BARROS, 0, TAIBA, SAO GONCALO D...",38.730.525/0001-55,BS PARC PACATUBA EMPREENDIMENTO IMOBILIARIO LTDA,"AV DOUTOR MENDEL STEINBRUCH,KM 12, - PAVUNA, P...","[{'codigo': '7.02', 'descricao': 'Servicos pre...",4400.00,"{'ICMS': None, 'IPI': None, 'ISS': 0.0, 'PIS':...",4.40,None,4.4
5,Teste-03.pdf,NOTA FISCAL DE SERVICO ELETRONICA - NFS-e\nFRA...,3,None,01/03/2023 07:35:01,None,Serviço,48.244.593/0001-68,FRANCISCO BRENO SANDRO GABRIEL BARROS,"R OSCAR SOARES BARROS, 0, TAIBA, SAO GONCALO D...",38.730.525/0001-55,BS PARC PACATUBA EMPREENDIMENTO IMOBILIARIO LTDA,"AV DOUTOR MENDEL STEINBRUCH,KM 12, - PAVUNA, P...","[{'codigo': '7.02', 'descricao': 'Servicos pre...",NaN,"{'ICMS': None, 'IPI': None, 'ISS': 0.0, 'PIS':...",4400.00,None,NaN
6,Teste-04 - Copia (10).PDF,RECEBI(EMOS) DE PET CENTER COMERCIO PARTICIPAC...,000.008.808,4,07-01-2025 12:49:34,2325 0118 3281 1801 3863 5500 4000 0088 0811 3...,Produto,18.328.118/0138-63,PET CENTER COMERCIO PARTICIPACOES S.A,"AV SANTOS DUMONT, 5650 COCO - FORTALEZA/CE CEP...",603.998.143-10,BRENO SANDRO,"RUA CONSELHEIRO LAFAYETTE, 200 JARDIM IRACEMA ...","[{'codigo': '7897683602231', 'descricao': 'FOC...",61.99,"{'ICMS': 13.64, 'IPI': 0.0, 'ISS': None, 'PIS'...",61.99,None,NaN
7,Teste-04 - Copia.PDF,RECEBI(EMOS) DE PET CENTER COMERCIO PARTICIPAC...,000.008.808,4,07-01-2025 12:49:34,23250118328118013863550040000088081138249364,Produto,18.328.118/0138-63,PET CENTER COMERCIO PARTICIPACOES S.A,"AV SANTOS DUMONT, 5650 COCO - FORTALEZA/CE CEP...",603.998.143-10,BRENO SANDRO,"RUA CONSELHEIRO LAFAYETTE, 200 JARDIM IRACEMA ...","[{'codigo': 

✅ Dados carregados e estruturados com sucesso!
